In [75]:
#This code computes first several equivariant Betti numbers of 
#isospectral manifold corresponding to a given graph.
#We assume that GKM-theory can be applied. 
#We use GKM theorem to describe equivariant cohomology graded-component-wise.
#In particular, it is assumed that manifold is equiv. formal.
#This leads to contradiction in some cases.

#variable ceiling stores how many homogeneous gradings of 
#equiv. cohomology should be computed.
ceiling=2
GroundField=QQ

#Initialize a graph. This one is Net
VertSet=[1,2,3,4,5,6]    
EdgeSet=[(1,2),(1,3),(2,3),(1,4),(2,4),(2,5),(3,5),(1,6),(3,6)]

In [76]:
# It is important that edges are defined as (i,j) not [i,j]. 
# Setting [i,j] leads to an error when treating an edge as a transposition afterwards. 

G=Graph([VertSet, EdgeSet], format='vertices_and_edges')

#n is the dimension of (noneffective) torus. 
#Here we work with noneffective action, since it is more convenient for GKM-theorem.
n=len(VertSet)

#2d is the real dimension of manifold.
d=len(EdgeSet)

In [77]:
#v0 stores complementary variable which will be specialized at edges. 
# v1,v2,... correspond to vertices of a graph G. 
#These are the generators of the polynomial algebra H*(BT)=Z[v1,...,vn] 

R = PolynomialRing(GroundField,n+1,"v")
v=R.gens()

In [78]:
#Here we initialize (unsigned) GKM-graph corresponding to our manifold. 
#All weights here have the form (0,...,0,1,0,..,0,-1,..,0) with 1 and -1 
#on some positions i,j, where (i,j) is the edge of G.
#We store the pair (i,j) as the label of edge of GKM-graph.

Sigma = SymmetricGroup(VertSet)
GKMedges = []
for sigma in Sigma :
    for e in EdgeSet :
        tau = sigma*Sigma(e)
        if Sigma.list().index(tau)>Sigma.list().index(sigma) :
            GKMedges.append([sigma, tau, e])

GKMgraph=Graph([Sigma, GKMedges], format='vertices_and_edges')

In [79]:
#Here we compute characteristics of GKM-graph
NumOfVert=len(GKMgraph.vertices())
NumOfEdges=len(GKMgraph.edges())

In [80]:
#We introduce the space, where our matrices live. 
#For simplicity (to forget signs), we work over Z_2.
IncidSpace = MatrixSpace(GroundField,NumOfEdges,NumOfVert)
IncidencePositions=[]

In [81]:
#This is essentially the incidence matrix of GKM-graph. 
#We often appeal to it, so it is better to compute it once.
#For each vertex we store the list of edges adjacent to that vertex. 
#Here by vertex|edge we mean their indices in the lists.
for i in range(NumOfVert) :
    vertex=GKMgraph.vertices()[i]
    edges=GKMgraph.edges_incident(vertex)
    curr = []
    for e in edges :
        r=list(GKMgraph.edges()).index(e)
        if e[0]==vertex :
            sign=1
        else :
            sign=-1
        curr.append([r,sign])
    IncidencePositions.append(curr)
    
for j in range(NumOfEdges) :
    edge=GKMgraph.edges()[j]
    IncidencePositionsTrans.append([edge[0],edge[1]])

In [82]:
#This function is the basic block of "Chang-Skjelbred-differential". 
#Its input is a monomial from a graded component of Z[v1,...,vn]. 
#It lands in a monomial from Z[v1,...,vn]/(weight).
#Since weight has the form (0,...,0,1,0,..,0,-1,..,0), 
#we simply substitute variables vi and vj with a single variable v0.

def Substitute(mono, pair) :
    w=[m for m in v]
    w[0]=1
    w[pair[0]]=v[0]
    w[pair[1]]=v[0]
    return mono(w)


In [83]:
def EquivBetti(degree) :
    #The collection of all monomials of degree degree in variables v1,...vn
    MonomialBasis=[R.monomial(*e) for e in WeightedIntegerVectors(degree,[ceiling+1]+[1]*n)]
    #The collection of all monomials of degree degree in var's v0,v1,...,vn
    #At this point one can do some optimization, but let's leave it this way for simplicity
    ExtendedBasis=[R.monomial(*e) for e in WeightedIntegerVectors(degree,[1]*(n+1))]

    d1=NumOfVert
    d2=len(MonomialBasis)

    t1=NumOfEdges
    t2=len(ExtendedBasis)
    
    #This dictionary stores a building block of Chang-Skjelbred differential
    SubstitutionPositions={}
    
    for e in EdgeSet :
        SubstitutionPositions[e]=[0]*t2
        for i2 in range(d2) :
            monom=MonomialBasis[i2]
            outputMonom=Substitute(monom,e)
            j2=ExtendedBasis.index(outputMonom)
            SubstitutionPositions[e][i2]=j2

    dTot=d1*d2
    tTot=t1*t2
    
    #print(dTot, tTot)

    Space = MatrixSpace(GroundField,tTot,dTot)
    
    #BFM stands for "extremely big matrix". 
    #This is the matrix of Chang-Skjelbred differential in the natural monomial basis.
    BFM = Space([0]*dTot*tTot)

    for i1 in range(d1) :
        #vert=GKMgraph.vertices()[i1]
        for j1a in IncidencePositions[i1] :
            j1=j1a[0]
            sign=j1a[1]
            e=GKMgraph.edges(labels=True)[j1]
            label=e[2]
            for i2 in range(d2) :
                j2=SubstitutionPositions[label][i2]
                i=i2*d1+i1
                j=j2*t1+j1
                BFM[j,i]=sign
       
    print("BFM initialized")
    Kernel=BFM.right_nullity()
    #It's better to print intermediate results as they appear, 
    #since calculations are time consuming
    
    print("degree=", degree, "EqDim =", Kernel)
    return Kernel

In [84]:
#Return to main program. We compute equiv. Betti numbers from 0-th till (ceiling-1)-st.
EquivBettis = [EquivBetti(degree) for degree in range(ceiling)]

SerCont.<t> = QQ[]
HilbEquiv=0
for i in range(len(EquivBettis)) :
    HilbEquiv+=(EquivBettis[i]*t^i)


BFM initialized
degree= 0 EqDim = 1
BFM initialized
degree= 1 EqDim = 5
BFM initialized
degree= 2 EqDim = 14
BFM initialized
degree= 3 EqDim = 29


In [85]:
#Here we compute ordinary Betti numbers under equiv.formality assumption. This means
# Hilb(H*)=Hilb(H*_T)*(1-t)^n
HilbOrd=HilbEquiv*(1-t)^n

print(HilbOrd)

-29*t^6 + 73*t^5 - 50*t^4 + t^3 + 2*t^2 + 2*t + 1
